# Import Modules

In [108]:
import pandas as pd
import os

# Preparing Directories
* Before running any other code, make sure output folders are created to help keep everything organized. 
* use os.listdir() to generate a list of the working directory to check if folders already exist 
* use os.mkdir() to make new folders

***tip: use an if statement to check if output folders already exist to avoid raising an error when you try to create a folder that already exists***

In [109]:
if 'ready_for_tb_import' not in os.listdir():
    os.mkdir('./ready_for_tb_import/')

if 'processed_quickbooks_files' not in os.listdir():
    os.mkdir('./processed_quickbooks_files/')

# Creating a Dictionary of Files to Format
## You will need:
1) An excel file with entity names that match the quickbooks file names (minus the .xlsx extension) and corresponding account suffixes to use for CCH Engagement
2) All of your quickbooks data collected into a single folder within your working directory

*if you need a refresher on why these suffixes are needed, see the project README file on github [Trial Balance Formatting](https://github.com/jacxson/Trial-Balance-Formatting)*

## Steps:
1) Create a dataframe from the excel file with entity names and suffixes (account_keys.xlsx)
2) Create a list of files in the folder that has the excel files exported from quickbooks (./quickbooks_data/)
3) Loop through the entities in the dataframe and compare it to the list of quickbooks files. Add any matches to a dictionary that contains the file name and the corresponding account suffix from the dataframe.



#### 1) Create dataframe of account_key.xlsx

In [110]:
entities_df = pd.read_excel('account_keys.xlsx')

# view first 5 rows
entities_df.head()

,Acronym,Trial Balance,Entity
0,ABC,34-ABC,ABC Subsidiary
1,DEF,34-DEF,DEF Subsidiary
2,GHI,34-GHI,GHI Subsidiary
3,JKL,34-GPD,JKL Subsidiary
4,MNO,34-MNO,MNO Subsidiary


#### 2. Create list of files in the quickbooks_data folder

In [111]:
data = os.listdir('./quickbooks_data/')

#### 3) Loop through the entities in the dataframe and compare it to the list of quickbooks files. Add any matches to a dictionary that contains the file name and the corresponding account suffix from the dataframe.

***tip: use the zip() function to loop through multiple dataframe columns simultaneously***

In [112]:
# Create an empty dictionary
entity_dict = {}

# use zip() function to loop through entity names and suffixes in the entities dataframe simulaneously
for entity, suffix in zip(entities_df['Entity'], entities_df['Acronym']):
    
    # if a match occurs between the files in the data list and the entities column, add the filename as key and the suffix as value to the empty dictionary
    if entity + '.xlsx' in data:
        entity_dict.update({f'{entity}.xlsx':f'.{suffix}'})
        
# view the entity_dict
entity_dict

{'ABC Subsidiary.xlsx': '.ABC',
 'DEF Subsidiary.xlsx': '.DEF',
 'GHI Subsidiary.xlsx': '.GHI'}

# Applying Formatting to a Single Excel File
## Steps:
1) Import a single excel file as a dataframe
2) Drop the 'TOTAL' row
3) Replace all null values with zeros
4) Split account names and numbers into new separate columns
5) Add suffixes to account numbers
6) Combine debit and credit columns into a single balance column
7) Export a dataframe of only account numbers, account names, and balances to a new excel file
8) Move processed files to the appropriate output folder

#### 1) Import a single excel file as a dataframe

**Notes:**
* QuickBooks exports typically have the default sheet of the workbook dedicated to tips on updating the report in excel. To access the data itself, you must specify sheet_name='Sheet1' as a keyword argument in the pandas.read_excel function, otherwise it will return an empty dataframe
* QuickBooks exports also typically have the table starting on row 6 with headers on row 5. Pass skiprows=4 as a keyword argument in oder to read in the data correctly. 

In [113]:
abc = pd.read_excel('./quickbooks_data/ABC Subsidiary.xlsx', sheet_name='Sheet1', skiprows=4)

# view first 5 rows of data
abc.head()

,Unnamed: 0,Unnamed: 1,Debit,Unnamed: 3,Credit
0,NaN,1010 · Example Bank - Business Banking,716.80,NaN,NaN
1,NaN,1210 · Cash In Transit,0.00,NaN,NaN
2,NaN,13010 · Land Held for Investment:13011 · Land ...,3561613.12,NaN,NaN
3,NaN,13010 · Land Held for Investment:13011 · Land ...,131260.00,NaN,NaN
4,NaN,13010 · Land Held for Investment:13011 · Land ...,48155.45,NaN,NaN


In [114]:
# view last 5 rows of data
abc.iloc[-5:]

,Unnamed: 0,Unnamed: 1,Debit,Unnamed: 3,Credit
25,NaN,20400 · Notes Payable:20401 · Note Payable - J...,NaN,NaN,1000000.00
26,NaN,60400 · Bank Service Charges,515.00,NaN,NaN
27,NaN,67300 · Management Fee Exp.,15400.00,NaN,NaN
28,NaN,69000 · Political Contribution,0.00,NaN,NaN
29,TOTAL,NaN,5879393.52,NaN,5879393.52


#### 2) Drop the 'TOTAL' row.

* The total row should be excluded from the trial balance import into CCH Engagement as the program will automatically calculate the total as a test of whether the trial balance is in balance. 

***tip: use an if statement to double check whether 'TOTAL' is the last column of the dataframe to avoid accidentally deleting a row of data***

In [115]:
# Check if last row is total, use .lower() to coerce all characters to be lowercase
if 'total' in str(abc.iloc[-1, 0]).lower():
    abc.drop(index=len(abc) - 1, inplace=True)

In [116]:
abc.iloc[-5:]

,Unnamed: 0,Unnamed: 1,Debit,Unnamed: 3,Credit
24,NaN,20100 · Due To/(From) Related Entities:20101 ·...,1043861.84,NaN,NaN
25,NaN,20400 · Notes Payable:20401 · Note Payable - J...,NaN,NaN,1000000.0
26,NaN,60400 · Bank Service Charges,515.00,NaN,NaN
27,NaN,67300 · Management Fee Exp.,15400.00,NaN,NaN
28,NaN,69000 · Political Contribution,0.00,NaN,NaN


#### 3) Replace all null values with zeros
To help avoid issues with adding and substracting later, we can go ahead and replace missing values with zeroes.

In [117]:
abc.fillna(0, inplace=True)

#### 4) Split account names and numbers into new separate columns
*This is the most invloved section of the notebook, and where the most time is saved*

1) Observe how account names and numbers are nested, and more importantly, how they are separated
2) Use .split() method with list comprehensions to select only account names and only account numbers
3) Create a function that makes it easier to apply splitting to make new columns

In [118]:
abc.head()

,Unnamed: 0,Unnamed: 1,Debit,Unnamed: 3,Credit
0,0,1010 · Example Bank - Business Banking,716.80,0.0,0.0
1,0,1210 · Cash In Transit,0.00,0.0,0.0
2,0,13010 · Land Held for Investment:13011 · Land ...,3561613.12,0.0,0.0
3,0,13010 · Land Held for Investment:13011 · Land ...,131260.00,0.0,0.0
4,0,13010 · Land Held for Investment:13011 · Land ...,48155.45,0.0,0.0


**Note:**
* Account names and numbers are separarated by " · "
* But the accounts are also listed nested inside of account groups and subgroups separated by ":"
* To complicate things further, some accounts do not have account groups and/or subgroups included (so we cannot just choose the same index every time)

**See the account at index 2 as an example:**
* The account name is 'Land' and the account number is '13011.1'
* The account sub-group is 'Land Basis' and the sub-group account number is '13011'
* The account group in 'Land Help for Investment' and the group account number is '13010'

***tip: to consistently select only 'Land' and '13011.1' whether or not the account group or subgroup is present, use a list comprehension that selects only the last element of each list of splits***

In [119]:
print(f"String before splitting: {abc['Unnamed: 1'][2]}\n")
print(f"List after splitting: {abc['Unnamed: 1'][2].split(':')}\n")
print(f"Last element of split list: {abc['Unnamed: 1'][2].split(':')[-1]}")

String before splitting: 13010 · Land Held for Investment:13011 · Land Basis:13011.1 · Land

List after splitting: ['13010 · Land Held for Investment', '13011 · Land Basis', '13011.1 · Land']

Last element of split list: 13011.1 · Land


In [120]:
# Create list comprehension of every last element of split lists and assign it to a new column
abc['accts_names'] = [x[-1] for x in abc['Unnamed: 1'].str.split(':')]

# View list of account names and numbers
abc['accts_names'].head()

0    1010 · Example Bank - Business Banking
1                    1210 · Cash In Transit
2                            13011.1 · Land
3                  13011.3 · Assignment Fee
4                      13012 · Closing Cost
Name: accts_names, dtype: object

***tip: since we can use essentially the same process to access the account names and numbers from the new column, we can write a function to make repeating this process easier.***

In [121]:
def new_col_from_split(df, split_col, delim, index=-1):
    
    return [x[index] for x in df[split_col].astype(str).str.split(delim)]

In [122]:
# Compare the first 5 outputs of our function to the first 5 lines of the acct_names column already created
new_col_from_split(abc, 'Unnamed: 1', delim=':')[:5]


['1010 · Example Bank - Business Banking',
 '1210 · Cash In Transit',
 '13011.1 · Land',
 '13011.3 · Assignment Fee',
 '13012 · Closing Cost']

In [123]:
# Apply the new function to the acct_names column (only viewing first 5 outputs)
abc[:5]

,Unnamed: 0,Unnamed: 1,Debit,Unnamed: 3,Credit,accts_names
0,0,1010 · Example Bank - Business Banking,716.80,0.0,0.0,1010 · Example Bank - Business Banking
1,0,1210 · Cash In Transit,0.00,0.0,0.0,1210 · Cash In Transit
2,0,13010 · Land Held for Investment:13011 · Land ...,3561613.12,0.0,0.0,13011.1 · Land
3,0,13010 · Land Held for Investment:13011 · Land ...,131260.00,0.0,0.0,13011.3 · Assignment Fee
4,0,13010 · Land Held for Investment:13011 · Land ...,48155.45,0.0,0.0,13012 · Closing Cost


**Note:**
* Since our 'index' parameter is set to a default of 0, this function will always grab the last item in each list unless instructed otherwise
* To access the account name, no index argument needs to be passed. To access account numbers, the index=0 argument will need to be passed

In [124]:
# Access account numbers wiht index=0 (only viewing first 5 outputs)
new_col_from_split(abc, 'accts_names', delim=' · ', index=0)[:5]

['1010', '1210', '13011.1', '13011.3', '13012']

In [125]:
# Assign account name and number columns to be equal to respective outputs of the new_col_from_split function
abc['account_number'] = new_col_from_split(abc, 'accts_names', delim=' · ', index=0)
abc['account_name'] = new_col_from_split(abc, 'accts_names', delim=' · ')

abc.head()

,Unnamed: 0,Unnamed: 1,Debit,Unnamed: 3,Credit,accts_names,account_number,account_name
0,0,1010 · Example Bank - Business Banking,716.80,0.0,0.0,1010 · Example Bank - Business Banking,1010,Example Bank - Business Banking
1,0,1210 · Cash In Transit,0.00,0.0,0.0,1210 · Cash In Transit,1210,Cash In Transit
2,0,13010 · Land Held for Investment:13011 · Land ...,3561613.12,0.0,0.0,13011.1 · Land,13011.1,Land
3,0,13010 · Land Held for Investment:13011 · Land ...,131260.00,0.0,0.0,13011.3 · Assignment Fee,13011.3,Assignment Fee
4,0,13010 · Land Held for Investment:13011 · Land ...,48155.45,0.0,0.0,13012 · Closing Cost,13012,Closing Cost


#### 5) Add suffixes to account numbers

In [126]:
# assign suffix variable to the value of the entity_dict belonging to the filename as key
suffix = entity_dict['ABC Subsidiary.xlsx']

# to avoid adding the suffix multiple time, check if the suffix is in 1 entry of the account numbers column before appending it to all of the account numbers
if suffix not in abc['account_number'][0]:
    abc['account_number'] += suffix

In [127]:
abc['account_number'].head()

0       1010.ABC
1       1210.ABC
2    13011.1.ABC
3    13011.3.ABC
4      13012.ABC
Name: account_number, dtype: object

#### 6) Combine debit and credit columns into a single balance column
The balance column can be easily created by subtracting the credit column from the debit column

In [128]:
abc['balance'] = abc['Debit'] - abc['Credit']

abc.head()

,Unnamed: 0,Unnamed: 1,Debit,Unnamed: 3,Credit,accts_names,account_number,account_name,balance
0,0,1010 · Example Bank - Business Banking,716.80,0.0,0.0,1010 · Example Bank - Business Banking,1010.ABC,Example Bank - Business Banking,716.80
1,0,1210 · Cash In Transit,0.00,0.0,0.0,1210 · Cash In Transit,1210.ABC,Cash In Transit,0.00
2,0,13010 · Land Held for Investment:13011 · Land ...,3561613.12,0.0,0.0,13011.1 · Land,13011.1.ABC,Land,3561613.12
3,0,13010 · Land Held for Investment:13011 · Land ...,131260.00,0.0,0.0,13011.3 · Assignment Fee,13011.3.ABC,Assignment Fee,131260.00
4,0,13010 · Land Held for Investment:13011 · Land ...,48155.45,0.0,0.0,13012 · Closing Cost,13012.ABC,Closing Cost,48155.45


#### 7) Export a dataframe of only account numbers, account names, and balances to a new excel file in the output folder
Trial Balance imports in CCH Engagement take only 3 columns: account numbers, account names, and balances.

In [129]:
# Assign account_number, account_name, and balance to a list that will serve as a list of columns to export
export_cols = ['account_number', 'account_name', 'balance']

#filter the dataframe by those columns
abc[export_cols].head()

,account_number,account_name,balance
0,1010.ABC,Example Bank - Business Banking,716.80
1,1210.ABC,Cash In Transit,0.00
2,13011.1.ABC,Land,3561613.12
3,13011.3.ABC,Assignment Fee,131260.00
4,13012.ABC,Closing Cost,48155.45


In [130]:
# set index = False to avoid exporting the indexes along with the data
abc[export_cols].to_excel('./ready_for_tb_import/formatted_tb_ABC Subsidiary.xlsx', index=False)

#### 8) Move processed files to the appropriate output folder

In [131]:
# Set filename, source folder, and destination folder variables
file = 'ABC Subsidiary.xlsx'
source_folder = './quickbooks_data/'
dest_folder = './processed_quickbooks_files/'

# Use os.rename to replace the source folder with the destination folder in the file path (use the if statement to avoid errors if you run this cell twice!)
if file in os.listdir(source_folder):    
    os.rename(source_folder + file, dest_folder + file)

# Putting it All Together: Refactoring
* To make this process scalable, we can write a few functions that allow us to loop through the files in the dictionary of entities and apply all of the transformations that were performed on 'ABC Subsidiary.xlsx' to every file in our source folder. 
* Once these functions are written, there is very little difference in the time it takes to format 1 trial balance or 100. 
* The use of functions also helps make this code easily adaptable to trial balance formatting for other companies. Quickbooks data is generally exported in similar formats, usually just with different delimiters between account names and numbers. Adapting this approach to another client is often as easy as replacing the delimiter in your function and the names of the source folders and running your script!

## 3 functions
1) The new_col_from_split function we already created
2) A function to create the dictionary of entities
3) A function to apply formatting


### 1) The new_col_from_split function we already created

Here we simply took the function defined earlier and added some explanation with a docstring.

In [132]:
def new_col_from_split(df, split_col, delim, index=-1):
    """This function takes a dataframe, a column to split, a delimiter, and an optional 
    index position that defaults to -1. It returns a list of the last element of each list that 
    results from splitting the split_col in the given dataframe (df). To select an element 
    other than the last element, use the index keyword argument to pass a different index position to select."""
    
    return [x[index] for x in df[split_col].astype(str).str.split(delim)]

#### 2) A function to create the dictionary of entities

Refactor the code used to create out entity_dict as a function

In [133]:
entities_df = pd.read_excel('account_keys.xlsx')

data = os.listdir('./quickbooks_data/')

entity_dict = {}

# use zip() function to loop through entity names and suffixes in the entities dataframe simulaneously
for entity, suffix in zip(entities_df['Entity'], entities_df['Acronym']):

    # if a match occurs between the files in the data list and the entities column, add the filename as key and the suffix as value to the empty dictionary
    if entity + '.xlsx' in data:
        entity_dict.update({f'{entity}.xlsx':f'.{suffix}'})

# view the entity_dict
entity_dict

{'DEF Subsidiary.xlsx': '.DEF', 'GHI Subsidiary.xlsx': '.GHI'}

**The necessary inputs are: a dataframe, the entity column of the dataframe, the suffix column of the dataframe, and the folder our data is stored in**

In [134]:
def create_entity_dict(dataframe, entity_column, suffix_column, data_folder='./quickbooks_data/'):
    """This function takes a dataframe, its entity column, its suffix column, and a 
    data_folder that default to quickbooks_data, and returns a dictionary of file names to format
    and suffixes for the account numbers in that file."""
    
    file_list = os.listdir(data_folder)
    entity_dict = {}
    for entity, suffix in zip(dataframe[entity_column], dataframe[suffix_column]):
        if entity + '.xlsx' in file_list:
            entity_dict.update({f'{entity}.xlsx': f'.{suffix}'})
            
    return entity_dict
            
        

**As long as we have a dataframe with entity names and suffixes, we can now pass any dataframe and its corresponding columns into the function, along with our data_folder if it is not "quickbooks_data" and create our entity dictionary**

In [135]:
entities_df = pd.read_excel('account_keys.xlsx')

dict_from_func = create_entity_dict(entities_df, 'Entity', 'Acronym')

dict_from_func

{'DEF Subsidiary.xlsx': '.DEF', 'GHI Subsidiary.xlsx': '.GHI'}

#### 3) A function to apply formatting
**For a reminder, the steps we applied to the single dataframe are listed again below with changes made in italics. We will add the same steps to our function, with additional step for looping through the entity_dict and creating folders if they are not present.**
1) Import a single excel file as a dataframe: ***assign the dataframe to a placeholder _df instead of abc, use f-string of datafolder and entity to reference the filename***
2) Drop the 'TOTAL' row: ***replace all occurances of abc with the placeholder _df***
3) Replace all null values with zeros: ***replace all occurances of abc with the placeholder _df***
4) Split account names and numbers into new separate columns ***combine with step 5***
5) Add suffixes to account numbers ***replace abc with _df, set new col name to placeholder _col, append the suffix to the account number as the list is created***
6) Combine debit and credit columns into a single balance column: ***replace abc with _df***
7) Export a dataframe of only account numbers, account names, and balances to a new excel file: ***replace abc with _df, use f-string to create filenames instead of hard keyed filenames.***
8) Move processed files to the appropriate output folder: ***use f-strings to reference the data_folder and filenames.***

In [136]:
def format_tbs(entities_dict, data_folder='./quickbooks_data/'):
    
    # new step: create folders if needed
    if 'ready_for_tb_import' not in os.listdir():
        os.mkdir('./ready_for_import/')
        
    if 'processed_quickbooks_files' not in os.listdir():
        os.mkdir('./processed_quickbooks_files/')
    
    # new step: loop through dictionary
    for entity, suffix in entities_dict.items():
        
        # new step: print filename currently being formatted to help if an error is thrown
        print(f'formatting {entity}...')
        
        # step 1: import file as dataframe
        _df = pd.read_excel(f'{data_folder}{entity}', sheet_name='Sheet1', skiprows=4)
        
        # step 2: drop unneeded 'TOTAL' row
        if 'total' in str(_df.iloc[-1, 0]).lower():
            _df.drop(index=len(_df) - 1, inplace=True)
            
        # step 3: replace null values with zeros
        _df.fillna(0, inplace=True)
        
        # step 4 and step 5: split account names and numbers into new separate columns and add suffixes to account numbers
        _df['_col'] = new_col_from_split(_df, 'Unnamed: 1', ':')
        _df['account_number'] = [account + suffix for account in new_col_from_split(_df, '_col', ' · ', index=0)]
        _df['account_name'] = new_col_from_split(_df, '_col', ' · ')
        
        # step 6: Combine debit and credit column
        _df['balance'] = _df['Debit'] - _df['Credit']
        
        #step 7: Export a dataframe of only account numbers, account names, and balances to a new excel file
        _df[['account_number', 'account_name', 'balance']].to_excel(f'./ready_for_tb_import/formatted_tb_{entity}', index=False)
        
        # step 8: Move processed files to the appropriate output folder
        os.rename(f'./{data_folder}/{entity}', f'./processed_quickbooks_files/{entity}')
        
        # new step: print statement to indicate that the file was formatted successfully
        print(f'formatted_tb_{entity} successfully created')

In [137]:
# pass our new formatting function on our dictionary created by our create_entity_dict function: dict_from_func
format_tbs(dict_from_func)

formatting DEF Subsidiary.xlsx...
formatted_tb_DEF Subsidiary.xlsx successfully created
formatting GHI Subsidiary.xlsx...
formatted_tb_GHI Subsidiary.xlsx successfully created


# Testing it Out
***It looks like the client just sent over quickbooks data for several more subsidiaries, could you set up a script to run using these functions on the files in the "more_quickbooks_data" folder***

### 1) Create your dataframe of entities and suffixes using (account_keys.xlsx)

In [138]:
ent_df = pd.read_excel('account_keys.xlsx')
ent_df.head()

,Acronym,Trial Balance,Entity
0,ABC,34-ABC,ABC Subsidiary
1,DEF,34-DEF,DEF Subsidiary
2,GHI,34-GHI,GHI Subsidiary
3,JKL,34-GPD,JKL Subsidiary
4,MNO,34-MNO,MNO Subsidiary


### 2) Create your dictionary of filenames and keys

In [139]:
ent_dict = create_entity_dict(ent_df, 'Entity', 'Acronym', data_folder='./more_quickbooks_data/')

ent_dict

{'JKL Subsidiary.xlsx': '.JKL',
 'MNO Subsidiary.xlsx': '.MNO',
 'PQR Subsidiary.xlsx': '.PQR',
 'STU Subsidiary.xlsx': '.STU',
 'VWX Subsidiary.xlsx': '.VWX',
 'YZ Subsidiary.xlsx': '.YZ',
 '123 Subsidiary.xlsx': '.123',
 '234 Subsidiary.xlsx': '.234',
 '345 Subsidiary.xlsx': '.345',
 '456 Subsidiary.xlsx': '.456',
 '567 Subsidiary.xlsx': '.567',
 '678 Subsidiary.xlsx': '.678',
 '789 Subsidiary.xlsx': '.789',
 '890 Subsidiary.xlsx': '.890',
 '999 Subsidiary.xlsx': '.999'}

### 3) Apply formatting

In [140]:
format_tbs(ent_dict, data_folder='./more_quickbooks_data/')

formatting JKL Subsidiary.xlsx...
formatted_tb_JKL Subsidiary.xlsx successfully created
formatting MNO Subsidiary.xlsx...
formatted_tb_MNO Subsidiary.xlsx successfully created
formatting PQR Subsidiary.xlsx...
formatted_tb_PQR Subsidiary.xlsx successfully created
formatting STU Subsidiary.xlsx...
formatted_tb_STU Subsidiary.xlsx successfully created
formatting VWX Subsidiary.xlsx...
formatted_tb_VWX Subsidiary.xlsx successfully created
formatting YZ Subsidiary.xlsx...
formatted_tb_YZ Subsidiary.xlsx successfully created
formatting 123 Subsidiary.xlsx...
formatted_tb_123 Subsidiary.xlsx successfully created
formatting 234 Subsidiary.xlsx...
formatted_tb_234 Subsidiary.xlsx successfully created
formatting 345 Subsidiary.xlsx...
formatted_tb_345 Subsidiary.xlsx successfully created
formatting 456 Subsidiary.xlsx...
formatted_tb_456 Subsidiary.xlsx successfully created
formatting 567 Subsidiary.xlsx...
formatted_tb_567 Subsidiary.xlsx successfully created
formatting 678 Subsidiary.xlsx...


In [142]:
# View formatted data
pd.read_excel('./ready_for_tb_import/formatted_tb_123 Subsidiary.xlsx')

,account_number,account_name,balance
0,1010.123,Example Bank - Business Banking,716.80
1,1210.123,Cash In Transit,0.00
2,13011.1.123,Land,3561613.12
3,13011.3.123,Assignment Fee,131260.00
4,13012.123,Closing Cost,48155.45
5,13013.123,Property Taxes,537.42
6,13061.123,Interest,245590.42
7,13064.123,Closing Cost,14600.00
8,13065.123,Origination Fee,39700.00
9,13022.123,Property Insurance,1252.95
